In [1]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
import sys

In [2]:
t1 = time.time()
# df_train = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_fold_6_in_6.csv')
df_train = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_fold_6_in_6_last0.2.csv')
t2 = time.time()
print('training loading done! Time: {}'.format(t2 - t1))
print('size is: {}'.format(sys.getsizeof(df_train) / 1024 ** 3))
print('length is {}'.format(len(df_train)))

training loading done! Time: 190.96250867843628
size is: 12.35285073518753
length is 6163463


In [3]:
# length = len(df_train)
# df_train2 = df_train.iloc[- int(length*0.2): ]
# df_train2.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_fold_6_in_6_last0.2.csv', index = False)

In [4]:
for i in df_train.columns:
    print(i)

ip
app
device
os
channel
day
hour
timestamp
minute
second
is_attributed
ip_count
ip_mean
app_count
app_mean
device_count
device_mean
os_count
os_mean
channel_count
channel_mean
day_count
day_mean
hour_count
hour_mean
minute_count
minute_mean
second_count
second_mean
ip_app_count
ip_app_mean
ip_device_count
ip_device_mean
ip_os_count
ip_os_mean
ip_channel_count
ip_channel_mean
ip_day_count
ip_day_mean
ip_hour_count
ip_hour_mean
ip_minute_count
ip_minute_mean
ip_second_count
ip_second_mean
app_device_count
app_device_mean
app_os_count
app_os_mean
app_channel_count
app_channel_mean
app_day_count
app_day_mean
app_hour_count
app_hour_mean
app_minute_count
app_minute_mean
app_second_count
app_second_mean
device_os_count
device_os_mean
device_channel_count
device_channel_mean
device_day_count
device_day_mean
device_hour_count
device_hour_mean
device_minute_count
device_minute_mean
device_second_count
device_second_mean
os_channel_count
os_channel_mean
os_day_count
os_day_mean
os_hour_count
os

# Specify Categorical Columns

In [5]:
categorical_col = ['ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'timestamp', 'minute', 'second']
target = 'is_attributed'
feature_cols = list(set(df_train.columns) - set([target]))

# just use last 1% as val:

In [6]:
length = len(df_train)
df_val = df_train.iloc[- int(0.05 * length) :]

# Construct lightgbm dataset

In [7]:
y_train = df_train[target].values
y_val = df_val[target].values

lgb_train = lgb.Dataset(df_train, y_train, feature_name = feature_cols, categorical_feature = categorical_col)
lgb_val = lgb.Dataset(df_val, y_val, feature_name = feature_cols, categorical_feature = categorical_col)

import gc
gc.collect()

0

In [8]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 200,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 6, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 300,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.8, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'bagging_fraction': 0.8, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'metric' : 'auc'
    }

model_3 = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[50]	valid_0's auc: 1
[100]	valid_0's auc: 1
[150]	valid_0's auc: 1
[200]	valid_0's auc: 1


In [14]:
importance = pd.Series(model_3.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
importance.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/output/importance_train_fold_6_in_6_last0.2.csv', index=False)

In [13]:
### load test
df_test = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_fold_6_in_6.csv')

In [19]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')

df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']


In [20]:
df_sub['is_attributed'] = model_3.predict(df_test)

In [24]:
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/train_fold_6_in_6_last0.2_0402_1.csv.gz', compression='gzip', index=False)

In [22]:
df_sub

,click_id,is_attributed
0,0,9.999987e-01
1,1,9.999987e-01
2,2,9.999986e-01
3,3,9.999987e-01
4,4,9.999988e-01
5,5,9.999987e-01
6,6,9.999987e-01
7,7,9.999987e-01
8,9,9.999986e-01
9,8,9.999987e-01
